# Initialization

In [2]:
# to install the packages used in this notebook run the following
#using Pkg
#Pkg.add(["FileIO","WAV","LibSndFile","FFTW","Plots","PlotlyJS","ORCA","Interact","WebIO"])
#using WebIO
#WebIO.install_jupyter_nbextension()

In [3]:
# load needed packages
import LibSndFile
using FileIO: load, save, loadstreaming, savestreaming
using WAV, FFTW, Plots, Interact

In [14]:
# enable plotlyjs backend for interactive plots
plotly(size=(800,400), lw = 2)

Plots.PlotlyBackend()

In [5]:
# helper function
# displays an html player for a given wav file in the notebook
audioplayer(filepath) = """
    <head>
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
    <title>Simple Test</title>
    </head>
    
    <body>
    <audio controls="controls" style="width:600px" >
      <source src="$filepath" type="audio/wav" />
      Your browser does not support the audio element.
    </audio>
    </body>
"""

audioplayer (generic function with 1 method)

# Nature of a simple wave

In [6]:
function wave(frequency,amplitude,fs,τ)
    #= 
    frequency: cycles per second
    amplitude: multiplier to the sin
    τ:         total time duration of the wave
    fs:        samples per second
    =#
    # the semicolon (;) in Julia is not obligatory but prevents value emmision aka printing
    xs = range(0, τ, length=τ*fs) .* (2pi * frequency);
    sin.(xs) .* amplitude # last value to be emmited is returned automatically
end

wave (generic function with 1 method)

In [15]:
w = let frequency = 1, amplitude = 5, fs = 1000, τ = 10
    wave(frequency,amplitude,fs,τ)
end
plot(w)

<!DOCTYPE html>
 
 
 Plots.jl

In [16]:
w = let frequency = 3, amplitude = 5, fs = 1000, τ = 10
    wave(frequency,amplitude,fs,τ)
end
plot(w)

<!DOCTYPE html>
 
 
 Plots.jl

In [8]:
# needs WebIO
#=
@manipulate for frequency = 1:10, amplitude=0:0.1:10, fs=10:1000, τ=1:10
    xs = range(0, τ, length=τ*fs);
    plot(xs, wave(frequency,amplitude,fs,τ))
end
=#

In [22]:
# more complicated wave
y(t) = @. cos(10t) + cos(30t) + cos(80t)

plot(y(0:0.01:2))

<!DOCTYPE html>
 
 
 Plots.jl

# Why you should be interested in the fourier transform? This is why.

In [25]:
plot(abs.(fft(y(0:0.01:10))), xlabel = "Frequency", ylabel = "Applitude", 
    title = "Asbolute value of the fft for a signal", lc = :red)

<!DOCTYPE html>
 
 
 Plots.jl

Using the Fourier Transform we were able to determine the frequencies and amplitudes from the signal. This is amazing! This means that even if we do not have any knowledge about how the original signal was created,  we are still able to determine its component's properties.  
  
See the gif below for an enlightening graphical representation of an application of fourier transform on a signal $f$.  

![](https://upload.wikimedia.org/wikipedia/commons/7/72/Fourier_transform_time_and_frequency_domains_%28small%29.gif)

# The Fourier Transform Equation


The equation of the Fourier Transform: $$\LARGE \hat{g}(t) = \int_{-\infty}^{\infty} \! g(t) * e^{-2\pi * i * t * f} \, \mathrm{d}t $$


At first look the above equation looks very daunting but let's take a moment to understand why it is they way it is. We will start by explaining each of its components: 
* $g(t)$ is a wave function exactly like waves we created earlier with the _wave_ function.  
* $\hat{g}(t)$ is the fourier transform of the wave function, it is convention to represent it using a _hat_ symbol.  
* $e^{2\pi*i}$, this is perhaps the strangest part of the equation, let's break it down. First of all, it is derived from Euler's identity $e^{i*x} = cos(x) + i*sin(x)$. Using this identity each point on a unit circle (a circle with radius = 1 and center = (0,0)) can be expressed in a single complex $e^{i*x}$ number instead of the coordinates $(x,y) = (cos(x), sin(x))$. When $x = 2\pi$ the identity describes the full length of the circle's circumference.  
* $t$ symbolizes time and it is needed in the expression $e^{2\pi*i}$ in order to add the concept of a wave's evolution through time. In order to understand this better see the gif bellow.    
* $f$ is the frequency of rotation, a higher frequency means more rotations per minute and a lower frequency means fewer rotations. A more intuitive way to think of this is as a multiplier to the time component. If $f$ is small then time increases slower and thus $e^{2\pi*i}$ takes longer to complete a full rotation. The opposite happens when you multiply time with a larger number like 2 or 3 etc.   
*$-$ the minus sine is added because often when dealing with rotation it is convention to think about rotating in the clockwise direction. If this was not added, rotation using Euler's identity would be counter clockwise because of the way it is defined.

![Alt Text](./Sine_curve_drawing_animation.gif)

# Applying FT to an audio file

In [53]:
y, fs, nbits, opt = wavread("./joe.wav");

In [54]:
[length(y) length(fft(y))]

1×2 Array{Int64,2}:
 70393  70393

In [84]:
plot(abs.(fft(y[1:1000])), xlabel = "Συχνότητα", ylabel = "Ένταση", 
    title = "Απόλυτη τιμή fft για το ηχητικό φάσμα", lc = :red, lw = 2)

<!DOCTYPE html>
 
 
 Plots.jl

# Implementing the Fourier Transform

discrete fft 

fast fourier transform

In [81]:
function simple_fft(y)
    ly = length(y)
    res = Array{Complex, 1}(undef, ly)
    for f in eachindex(y) # frequency
        # (1:ly) and not eachindex because @. macro would broadcast
        # funcion calls like length or eachindex
        res[f] = sum(@. y * exp(-im * 2pi * f * (1:ly) / ly))
    end
    reverse(res)
end

simple_fft2(y) = (ly = length(y);
            [sum(@. y * exp(-im * 2pi * f * (1:ly) / ly)) for f in eachindex(y)] |> reverse)

simple_fft2 (generic function with 1 method)

# FFTSHIFT

In [ ]:
F = fft(w3) |> fftshift
freqs = fftfreq(length(w3), fs) |> fftshift

time_domain = plot(range(0,τ,length=τ*fs), w3, title = "Signal")
freq_domain = plot(freqs, abs.(F), title = "Spectrum", xlim=(-10, +10)) 
plot(time_domain, freq_domain, layout = 2, size=(900,400))

In [ ]:
fft(w3) |> fftshift .|> abs |>  plot

In [85]:
nŷ = simple_fft2(y[1:1000]);
plot(abs.(nŷ), xlabel = "Συχνότητα", ylabel = "Ένταση", 
    title = "Απόλυτη τιμή fft για το ηχητικό φάσμα", lc = :red, lw = 2)

<!DOCTYPE html>
 
 
 Plots.jl

# The Fourier Transform applied to a simple wave

In [ ]:
#fft(w) .|> (x->sqrt(real(x)^2 + imag(x)^2)) |> plot

In [ ]:
#fftfreq(length(w),fs) |> plot

In [ ]:
F = fft(w) |> fftshift
freqs = fftfreq(length(w), fs) |> fftshift

time_domain = plot(range(0,τ,length=τ*fs), w, title = "Signal")
freq_domain = plot(freqs, abs.(F), title = "Spectrum", xlim=(-10, +10)) 
plot(time_domain, freq_domain, layout = 2, size=(900,400))

# Real Life example of FFT

In [ ]:
display("text/html", audioplayer("./audio.wav"))

In [ ]:
y, fs, nbits, opt = wavread("./audio.wav");

In [ ]:
size(y), fs, nbits

196800 points, 48K metrhseis / sec

In [ ]:
nbits, typeof(nbits)

In [ ]:
# calculate duration of the audio clip
size(y)[1] / fs

4.1 deuterolepta

In [ ]:
times = collect(1:length(y)) / fs;

In [ ]:
plot(times,y)

In [ ]:
#=
yf = fft(y)

fft(y) .|> (x->sqrt(real(x)^2 + imag(x)^2)) |> plot

yif = ifft(y)

wavwrite(yif, "./ifft_audio.wav")

fft(y) .|> (x->sqrt(real(x)^2 + imag(x)^2)) |> histogram
=#